# Setup

In [ ]:
!pip install -q kaggle
!pip install -q ydata_profiling
!pip install -q imbalanced-learn

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"saatvikmtech","key":"e675c36acec415887dcabdbf052957ab"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d gauravduttakiit/smoker-status-prediction-using-biosignals
!unzip smoker-status-prediction-using-biosignals.zip

Dataset URL: https://www.kaggle.com/datasets/gauravduttakiit/smoker-status-prediction-using-biosignals
License(s): apache-2.0
smoker-status-prediction-using-biosignals.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  smoker-status-prediction-using-biosignals.zip
replace test_dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test_dataset.csv        
replace train_dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train_dataset.csv       


In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport

In [ ]:
df = pd.read_csv('train_dataset.csv')
df

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,35,170,85,97.0,0.9,0.9,1,1,118,78,...,70,142,19.8,1,1.0,61,115,125,1,1
1,20,175,110,110.0,0.7,0.9,1,1,119,79,...,71,114,15.9,1,1.1,19,25,30,1,0
2,45,155,65,86.0,0.9,0.9,1,1,110,80,...,57,112,13.7,3,0.6,1090,1400,276,0,0
3,45,165,80,94.0,0.8,0.7,1,1,158,88,...,46,91,16.9,1,0.9,32,36,36,0,0
4,20,165,60,81.0,1.5,0.1,1,1,109,64,...,47,92,14.9,1,1.2,26,28,15,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38979,40,165,60,80.0,0.4,0.6,1,1,107,60,...,61,72,12.3,1,0.5,18,18,21,1,0
38980,45,155,55,75.0,1.5,1.2,1,1,126,72,...,76,131,12.5,2,0.6,23,11,12,0,0
38981,40,170,105,124.0,0.6,0.5,1,1,141,85,...,48,138,17.1,1,0.8,24,23,35,1,1
38982,40,160,55,75.0,1.5,1.5,1,1,95,69,...,79,116,12.0,1,0.6,24,20,17,0,1


# Preprocessing

In [ ]:
print(df.duplicated().sum())
df.drop_duplicates(inplace=True)

5517


In [ ]:
x = df.drop(columns=['smoking'])
y = df['smoking']

In [ ]:
x[['hearing(left)', 'hearing(right)']] = x[['hearing(left)','hearing(right)']] - 1

x = pd.get_dummies(x, columns=['Urine protein'], prefix='urine_protein')

In [ ]:
x

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,...,AST,ALT,Gtp,dental caries,urine_protein_1,urine_protein_2,urine_protein_3,urine_protein_4,urine_protein_5,urine_protein_6
0,35,170,85,97.0,0.9,0.9,0,0,118,78,...,61,115,125,1,True,False,False,False,False,False
1,20,175,110,110.0,0.7,0.9,0,0,119,79,...,19,25,30,1,True,False,False,False,False,False
2,45,155,65,86.0,0.9,0.9,0,0,110,80,...,1090,1400,276,0,False,False,True,False,False,False
3,45,165,80,94.0,0.8,0.7,0,0,158,88,...,32,36,36,0,True,False,False,False,False,False
4,20,165,60,81.0,1.5,0.1,0,0,109,64,...,26,28,15,0,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38978,40,170,65,77.0,1.5,1.5,0,0,110,62,...,28,43,36,1,True,False,False,False,False,False
38979,40,165,60,80.0,0.4,0.6,0,0,107,60,...,18,18,21,1,True,False,False,False,False,False
38980,45,155,55,75.0,1.5,1.2,0,0,126,72,...,23,11,12,0,False,True,False,False,False,False
38982,40,160,55,75.0,1.5,1.5,0,0,95,69,...,24,20,17,0,True,False,False,False,False,False


# Models

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [ ]:
categorical_cols = ['hearing(left)', 'hearing(right)', 'Urine protein', 'dental caries', 'smoking']
continuous_cols = [col for col in df.columns if col not in categorical_cols]

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
pp = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), continuous_cols)
    ],
    remainder='passthrough'
)

## Logistic Regression

In [ ]:
lrpipe = Pipeline([
    ('preprocess', pp),
    ('clf', LogisticRegression(max_iter=10000))
])

lrparam = {
    'clf__penalty': [None, 'elasticnet', 'l1', 'l2', ],
    'clf__C': [0.1, 0.3, 1, 3, 10, 30],
    'clf__l1_ratio': [0.25, 0.5, 0.75],
    'clf__solver': ['saga', 'lbfgs']
}

In [ ]:
gridlr = GridSearchCV(estimator=lrpipe, param_grid=lrparam, scoring='accuracy', cv=skf, verbose=3, return_train_score=True)
gridlr.fit(x, y)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=None)
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1207: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


[CV 1/5] END clf__C=0.1, clf__l1_ratio=0.25, clf__penalty=None;, score=(train=0.721, test=0.724) total time=   0.4s


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=None)
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1207: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


[CV 2/5] END clf__C=0.1, clf__l1_ratio=0.25, clf__penalty=None;, score=(train=0.723, test=0.717) total time=   0.3s


...

[CV 4/5] END clf__C=30, clf__l1_ratio=0.75, clf__penalty=l2;, score=(train=0.725, test=0.713) total time=   0.4s


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


[CV 5/5] END clf__C=30, clf__l1_ratio=0.75, clf__penalty=l2;, score=(train=0.722, test=0.726) total time=   0.9s
[CV 1/5] END clf__C=30, clf__l1_ratio=0.75, clf__penalty=elasticnet;, score=(train=nan, test=nan) total time=   0.0s
[CV 2/5] END clf__C=30, clf__l1_ratio=0.75, clf__penalty=elasticnet;, score=(train=nan, test=nan) total time=   0.0s
[CV 3/5] END clf__C=30, clf__l1_ratio=0.75, clf__penalty=elasticnet;, score=(train=nan, test=nan) total time=   0.0s
[CV 4/5] END clf__C=30, clf__l1_ratio=0.75, clf__penalty=elasticnet;, score=(train=nan, test=nan) total time=   0.0s
[CV 5/5] END clf__C=30, clf__l1_ratio=0.75, clf__penalty=elasticnet;, score=(train=nan, test=nan) total time=   0.0s


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
180 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, **

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['age',
                                                                          'height(cm)',
                                                                          'weight(kg)',
                                                                          'waist(cm)',
                                                                          'eyesight(left)',
                                                                          'eyesight(right)',
                                                                          'systolic',
                                                                          'relaxation',
                                                                          'fasting '
                                                                          'blood '
                                                                          'sugar',
                                                                          'Cholesterol',
                                                                          'triglyceride',
                                                                          'HDL',
                                                                          'LDL',
                                                                          'hemoglobin',
                                                                          'serum '
                                                                          'creatinine',
                                                                          'AST',
                                                                          'ALT',
                                                                          'Gtp'])])),
                                       ('clf',
                                        LogisticRegression(max_iter=10000))]),
             param_grid={'clf__C': [0.1, 0.3, 1, 3, 10, 30],
                         'clf__l1_ratio': [0.25, 0.5, 0.75],
                         'clf__penalty': [None, 'l1', 'l2', 'elasticnet']},
             return_train_score=True, scoring='accuracy', verbose=3)

In [ ]:
print(gridlr.best_score_)
print(gridlr.best_params_)

bestlr = gridlr.best_estimator_
ypred = bestlr.predict(x)
print(accuracy_score(y, ypred))
print(classification_report(y, ypred))
print(confusion_matrix(y, ypred))

0.7214270884264697
{'clf__C': 0.1, 'clf__l1_ratio': 0.25, 'clf__penalty': 'l2'}
0.7223533630143126
              precision    recall  f1-score   support

           0       0.76      0.81      0.79     21209
           1       0.64      0.56      0.60     12258

    accuracy                           0.72     33467
   macro avg       0.70      0.69      0.69     33467
weighted avg       0.72      0.72      0.72     33467

[[17273  3936]
 [ 5356  6902]]


## Support Vector Classifier

In [ ]:
svcpipe = Pipeline([
    ('preprocess', pp),
    ('clf', SVC(max_iter=100000))
])

svcparam = {
    'clf__C': [0.1, 0.3, 1, 3, 10],
    'clf__kernel': ['linear', 'poly', 'sigmoid', 'rbf']
}

In [ ]:
gridsvc = GridSearchCV(estimator=svcpipe, param_grid=svcparam, scoring='accuracy', cv=skf, verbose=3, return_train_score=True)
gridsvc.fit(x, y)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END clf__C=0.1, clf__kernel=linear;, score=(train=0.728, test=0.732) total time=  20.9s
[CV 2/5] END clf__C=0.1, clf__kernel=linear;, score=(train=0.730, test=0.725) total time=  21.2s
[CV 3/5] END clf__C=0.1, clf__kernel=linear;, score=(train=0.727, test=0.734) total time=  21.7s
[CV 4/5] END clf__C=0.1, clf__kernel=linear;, score=(train=0.733, test=0.716) total time=  20.7s
[CV 5/5] END clf__C=0.1, clf__kernel=linear;, score=(train=0.728, test=0.731) total time=  21.6s
[CV 1/5] END clf__C=0.1, clf__kernel=poly;, score=(train=0.710, test=0.703) total time=  22.5s
[CV 2/5] END clf__C=0.1, clf__kernel=poly;, score=(train=0.711, test=0.707) total time=  22.0s
[CV 3/5] END clf__C=0.1, clf__kernel=poly;, score=(train=0.712, test=0.703) total time=  22.3s
[CV 4/5] END clf__C=0.1, clf__kernel=poly;, score=(train=0.717, test=0.694) total time=  22.2s
[CV 5/5] END clf__C=0.1, clf__kernel=poly;, score=(train=0.711, test=0.69

/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV 1/5] END clf__C=0.3, clf__kernel=linear;, score=(train=0.728, test=0.731) total time=  26.2s


/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV 2/5] END clf__C=0.3, clf__kernel=linear;, score=(train=0.729, test=0.724) total time=  25.9s


...

[CV 4/5] END clf__C=10, clf__kernel=rbf;, score=(train=0.819, test=0.738) total time=  54.2s


/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV 5/5] END clf__C=10, clf__kernel=rbf;, score=(train=0.821, test=0.739) total time=  59.2s


/usr/local/lib/python3.12/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['age',
                                                                          'height(cm)',
                                                                          'weight(kg)',
                                                                          'waist(cm)',
                                                                          'eyesight(left)',
                                                                          'eyesight(right)',
                                                                          'systolic',
                                                                          'relaxation',
                                                                          'fasting '
                                                                          'blood '
                                                                          'sugar',
                                                                          'Cholesterol',
                                                                          'triglyceride',
                                                                          'HDL',
                                                                          'LDL',
                                                                          'hemoglobin',
                                                                          'serum '
                                                                          'creatinine',
                                                                          'AST',
                                                                          'ALT',
                                                                          'Gtp'])])),
                                       ('clf', SVC(max_iter=100000))]),
             param_grid={'clf__C': [0.1, 0.3, 1, 3, 10],
                         'clf__kernel': ['linear', 'poly', 'sigmoid', 'rbf']},
             return_train_score=True, scoring='accuracy', verbose=3)

In [ ]:
print(gridsvc.best_score_)
print(gridsvc.best_params_)

bestsvc = gridsvc.best_estimator_
ypred = bestsvc.predict(x)
print(accuracy_score(y, ypred))
print(classification_report(y, ypred))
print(confusion_matrix(y, ypred))

0.7501116779340071
{'clf__C': 1, 'clf__kernel': 'rbf'}
0.7759285266083007
              precision    recall  f1-score   support

           0       0.82      0.83      0.82     21209
           1       0.70      0.68      0.69     12258

    accuracy                           0.78     33467
   macro avg       0.76      0.76      0.76     33467
weighted avg       0.77      0.78      0.78     33467

[[17637  3572]
 [ 3927  8331]]


## Dense Neural Network

In [ ]:
nnskf = StratifiedKFold(n_splits=3, shuffle=False)

nnpipe = Pipeline([
    ('preprocess', pp),
    ('clf', MLPClassifier(solver='adam', batch_size=8192, tol=0.1, early_stopping=True, random_state=42, validation_fraction=0.25))
])

nnparam = {
    'clf__learning_rate_init': [0.001, 0.003, 0.01, 0.03, 0.1, 0.3],
    'clf__alpha': [0, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3],
    'clf__hidden_layer_sizes': [(64,), (128,), (64, 64), (128, 128)],
    'clf__activation': ['logistic', 'tanh', 'relu'],
}

In [ ]:
gridnn = GridSearchCV(estimator=nnpipe, param_grid=nnparam, cv=nnskf)
gridnn.fit(x, y)

/usr/local/lib/python3.12/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['age',
                                                                          'height(cm)',
                                                                          'weight(kg)',
                                                                          'waist(cm)',
                                                                          'eyesight(left)',
                                                                          'eyesight(right)',
                                                                          'systolic',
                                                                          'relaxation',
                                                                          'fasting '
                                                                          'blood '
                                                                          'sugar',
                                                                          'Cholesterol',
                                                                          'triglycerid...
                                                                          'AST',
                                                                          'ALT',
                                                                          'Gtp'])])),
                                       ('clf',
                                        MLPClassifier(batch_size=8192,
                                                      early_stopping=True,
                                                      random_state=42, tol=0.1,
                                                      validation_fraction=0.25))]),
             param_grid={'clf__activation': ['logistic', 'tanh', 'relu'],
                         'clf__alpha': [0, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3],
                         'clf__hidden_layer_sizes': [(64,), (128,), (64, 64),
                                                     (128, 128)],
                         'clf__learning_rate_init': [0.001, 0.003, 0.01, 0.03,
                                                     0.1, 0.3]})

In [ ]:
print(gridnn.best_score_)
print(gridnn.best_params_)

bestnn = gridnn.best_estimator_
ypred = bestnn.predict(x)
print(accuracy_score(y, ypred))
print(classification_report(y, ypred))
print(confusion_matrix(y, ypred))

0.7427017449236147
{'clf__activation': 'relu', 'clf__alpha': 0.1, 'clf__hidden_layer_sizes': (128,), 'clf__learning_rate_init': 0.03}
0.7509486957301222
              precision    recall  f1-score   support

           0       0.80      0.82      0.81     21209
           1       0.67      0.64      0.65     12258

    accuracy                           0.75     33467
   macro avg       0.73      0.73      0.73     33467
weighted avg       0.75      0.75      0.75     33467

[[17318  3891]
 [ 4444  7814]]
